In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
dfT = pd.read_csv('./data/thyroidDF.csv')
dfT.drop(columns=['patient_id'], inplace=True)

In [ ]:
dfT.shape, dfT.columns

# indices que são numéricos
# 0 17 19 21 23 25 27

In [ ]:
# Esse valor tem os dados desbalanceados devido uma quantidade de NaN presente
# E por termos valores vazios no dataset

dfT.mean()

In [ ]:
dfT.describe()

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(dfT.corr(), annot=True, cmap="magma");

In [ ]:
sns.countplot(dfT['age'], palette='mako');
# dfT[dfT['age'] <= 0] >>>> sem idades negativas

In [ ]:
plt.hist(x = dfT['age'], color='lightgreen');

In [ ]:
# praticamente todas as mulheres do banco de dados estavam grávidas durante as análises
sns.countplot(dfT['pregnant'], palette='mako');

In [ ]:
sns.countplot(dfT['sex'], palette='mako');

In [ ]:
# Thyroxine-binding globulin (TBG), X[:,6] is one of three major transport proteins, which are primarily 
# responsible for binding to and transporting thyroid hormones to the necessary tissues

f, ax = plt.subplots(figsize=(8, 8))
sns.scatterplot(data=X, x=X[:, 0], y=X[:, 6], s=25, color='red');

In [ ]:
X = dfT.iloc[:, [0, 17, 19, 21, 23, 25, 27]].values
X

In [ ]:
X

In [ ]:
X[:,0].min(), X[:,0].max()

In [ ]:
X[:, 0].min(), X[:, 0].max()

In [ ]:
y = dfT.iloc[:, 29].values
y

In [ ]:
# Escalonamento das variáeis
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X = scalar.fit_transform(X)

In [ ]:
X

In [ ]:
## Label Enconder para tratamento de variáveis categóricas
from sklearn.preprocessing import LabelEncoder

In [ ]:
dfT_X = dfT.iloc[:, 0:29].values

In [ ]:
label_enconder_sex = LabelEncoder()
label_encoder_TBG = LabelEncoder()

In [ ]:
dfT_X[:,1] = label_enconder_sex.fit_transform(dfT_X[:,1])
dfT_X[:,27]= label_encoder_TBG.fit_transform(dfT_X[:,27])

In [ ]:
dfT_X